# Øving 10 - Maksimal flyt

In [ ]:
include("checker.jl")
include("flow_checker.jl")
include("theory.jl")
import FlowChecker
import TheoryChecker

In [ ]:
theory_answers = Dict{Int, TheoryChecker.IntOrFloatOrString}([]);

## Teori


Q1: Hva er maksimal flyt-problemet?

1. Kvantifisere hvor mye bedre eller værre enn suboptimal løsning av korteste-vei problemet er
2. Å maksimere flyten mellom to noder i en graf
3. Å finne en flyt av maksimal flyt-verdi gjennom et nettverk

In [ ]:
theory_answers[1] = 1;

Q2: Hva betyr $c(v_1, v_2) = 10$, $f(v_1, v_2) = 3$ ?
1. Enhver flyt fra $v_1$ til $v_2$ koster 10. Flyten er 3, og den totale kostnaden fra $v_1$ til $v_2$ blir 30.
2. Kapasiteten fra $v_2$ til $v_1$ er 10. Nåværende flyt fra $v_2$ til $v_1$ er 3.
3. Kapasiteten fra $v_1$ til $v_2$ er 10. Nåværende flyt fra $v_1$ til $v_2$ er 3.

In [ ]:
theory_answers[2] = 1;

Q3: Hva er flyten $f(v_2, v_3)$ i dette nettverket?

In [ ]:
draw_network(G, E, F, C)

1. 0
2. 2
3. -2
4. 8

In [ ]:
theory_answers[3] = 1;

Q4: Hva er galt med dette nettverket? 

In [ ]:
draw_network(G, E2, F2, C2)

1. Flyten er ikke opprettholdt
2. Det må være flyt i alle kanter i ett flytnettverk
3. Det går for liten flyt mellom node 1 og 3 ($f(v_1, v_3)$)

In [ ]:
theory_answers[4] = 1;

In [ ]:
draw_network(G, E3, F3, C3)

Q5: Iflg Edmonds-Karps algoritme, hvilken sti vil være den neste flytøkende stien?


1. `[1 2 4]`
2. `[1 3 2 4]`
3. `[1 2 3 4]`

In [ ]:
theory_answers[5] = 1;

In [ ]:
include("practice.jl")

## Praksis

Her skal du løse maksimal-flyt problemet. Hvilken algoritme du implementerer er opp til deg, men Edmonds-Karp eller Ford-Fulkerson(s. 724 i Cormen) er gode kandidater. Det er  allerede satt opp ett rammeverk til deg.

### Rammeverk
$G$ er den underliggende grafen.

$E$ gir uttrykk for $c(\cdotp ,\cdotp)$ i $G$.<br>
Eksempelvis:
```julia
E[1] = [1 2 16]
```
Dette betyr at det går en kant fra node 1 til node 2 med kapasitet på 16.

$F$ er en naboskapsmatrise, og gir uttrukk for $f(\cdotp, \cdotp)$ i $G$.

Enhver indeks i $F$ gir altså flyten fra indeksen til alle de andre nodene i grafen.<br>
I starten vil det ikke gå noe flyt gjennom grafen, og alle indekser i $F$ vil være slike:
```julia
F[1] = [0 0 0 0]
```

$C$ er, i likhet med $F$ en naboskapsmatrise, og gir uttrykk for $c(\cdotp, \cdotp)$ i $G$.

```julia
C[1] = [0 16 13 0 0 0]
```

In [ ]:
draw_initial_network()

Her skal du fullføre funksjonen for å finne en flytforøkende sti i en graf. Funksjonen er nesten ferdig, og trenger noen linjer kode der kommentaren er.

På grafen over skal den returnere [1 2 4 6] eller [1 3 5 6].

In [ ]:
function find_aug_path(G, E, F, C, source, sink)
    Q = Queue(Int)
    V = falses(nv(G))
    P = Dict{Int, Int}()

    function get_path()
        n = sink
        res = []
        while n != 0
            push!(res, n)
            n = P[n]
        end
        reverse!(res)
    end

    enqueue!(Q, source)
    P[source] = 0
    V[source] = true

    while length(Q) > 0
        n = dequeue!(Q)
        if n == sink
            return get_path()
        end
        for nb in all_neighbors(G, n)
            # Sjekk om naboen er med i den flytforøkende stien
            # START IKKE UTDELT
            if !V[nb] && F[n, nb] < C[n, nb]
                    enqueue!(Q, nb)
                    V[nb] = true
                    P[nb] = n
                end
            # SLUTT IKKE UTDELT
        end
    end
    []
end
draw_network(G, E, F)
p = find_aug_path(G, E, F, C, 1, 6)
println(p)

Her skal du løse maksimal-flyt problemet. `find_aug_path()` brukes til å finne den nye flytforøkende stien. Funksjonen `max_flow(G, E, F)` skal returnere den maksimale flyten $|\ f\ |$ i grafen $G$.

In [ ]:
draw_network(G,E,F)
function max_flow(G, E, C, source, sink, draw=false)
    
    F = zeros(C)
    #find augmenting path
    path = find_aug_path(G, E, F, C, source, sink)
    while length(path) > 0
        f = typemax(Int)
        for i in 1:length(path)-1
            # Finn maksimal flyt som kan gå gjennom denne stien
            # START IKKE UTDELT KODE
            u, v = path[i], path[i+1]
            f = min(f, C[u, v] - F[u, v])
            # SLUTT IKKE UTDELT KODE
            
        end
        for i in 1:length(path)-1
            # Oppdater flyten langs stien her
            # START IKKE UTDELT KODE
            u, v = path[i], path[i+1]
            F[u, v] += f
            # SLUTT IKKE UTDELT KODE
            
        end
        path = find_aug_path(G, E, F, C, source, sink)
        if (draw)
            draw_network(G,E,F)
        end
    end
    
    # Regn ut flyten
    flow = 0
    for u = 1:nv(G)
        flow += F[u, sink]
    end
    
    return flow
end
max_flow(G, E, C, 1, 6, true);

**Teori**: Linja under sjekker om teorispørsmålene har gyldige verdier.

In [ ]:
TheoryChecker.check_answers(theory_answers, FlowChecker.answer_metadata);

**Praksis**: Linja under kjører noen lokale testtilfeller for find_aug_path-funksjonen.

In [ ]:
CodeChecker.check_answers(find_aug_path, FlowChecker.test_augpath_input, FlowChecker.test_augpath_expected_output)

**Praksis**: Linja under kjører noen lokale testtilfeller for max_flow-funksjonen.

In [ ]:
CodeChecker.check_answers(max_flow, FlowChecker.test_maxflow_input, FlowChecker.test_maxflow_expected_output)